In [1]:
import pandas as pd
import numpy as np
import copy
import os

In [35]:
def hay_tabla(archivo,verbose=False):
    '''
    Funcion para verificar si en la pagina (archivo) dada existe una tabla
    Si es el caso devolver la tabla
    '''
    try:
        if verbose:
            print('Hay tabla', archivo)
        df = pd.read_csv(archivo, header=1).transpose()
        df.reset_index(inplace=True)
        df.columns = df.iloc[0]
        return df.drop(index=[0])
    except Exception as e:
        return 'No hay tabla'

In [36]:
def corregir_santander(df):
    '''
    Funcion para dar formato a los trimestres de santander
    '''
    cols = df.columns
    annos = [i for i in df[cols[0]].values if not 'Unnamed' in i]
    tris = [i for i in df[cols[1]].values if not 'Unnamed' in i]
    cont = -1
    correciones = []
    for i in range(len(tris)):
        if tris[i] == '6M':
            cont += 1
        correciones.append(f'{tris[i]}{annos[cont][-2:]}')
    cols = list(df.columns)
    cols[0] = 'periodos'
    df.columns = cols
    df[list(df.columns)[0]] = correciones
    cols.remove(cols[1])
    df = df[cols]
    df = df.loc[[not ('6M' in i) for i in list(df['periodos'])]]
    return df

In [37]:
def corregir_bbva(df):
    '''
    Funcion para dar formato a los trimestres de bbva
    '''
    cols = df.columns
#     print(df.head())
    df[cols[0]] = df[cols[0]].apply(lambda x: str(x)[:str(x).find('T')+1]+str(x)[-2:])
#     print(df.head())
    return df

In [38]:
def merge_data(df1, df2):
    '''
    Anadimos los elementos que existen en un data frame a otro sin sobreescribir los que estan en ambos
    '''
    for i in range(len(df1)):
        if not df1[list(df1.columns)[0]].iloc[i] in list(df2[list(df2.columns)[0]].values):
            df2 = df2.append(df2.iloc[i])
    return df2

In [39]:
def m_f(keyword, keyword_black_list, banco, white_list=None, verbose=False):
    ## Es importante sortear whitelist por la forma en la que se hace join a los dataframes
    white_list.sort()
    cont = 0
    while len(white_list):
#         if :
            if cont == 0:
                df = all_in_one(keyword, keyword_black_list, banco, [white_list[0]], verbose)
                cont += 1
            else:
                temp = all_in_one(keyword, keyword_black_list, banco, [white_list[0]],verbose)
#                 print('AAAAAAAAAAA')
                df = merge_data(temp, df)
            white_list.remove(white_list[0])
    if banco == 'Santander':
        df = corregir_santander(df)
    elif banco == 'BBVA':
        df = corregir_bbva(df)
    cols = list(df.columns)
    cols[0] = 'periodos'
    df.columns = cols
    return df

In [40]:
def all_in_one(keyword, keyword_black_list, banco, white_list=None, verbose=False):
    d = encontrar_keyword_banco(keyword, keyword_black_list, banco, white_list,verbose)
    ## Nos aseguramos de que no haya tablas vacias
    d = {k: v for k, v in d.items() if v}
    df = procesar_tablas(d, banco)
    df['Banco'] = banco
    return limpiar_tabla(df.reset_index(drop=True))

In [42]:
def encontrar_keyword_banco(keyword, keyword_black_list, banco, white_list,verbose=False):
    '''
    Funcion para buscar las paginas coincidentes con las keyword en todos los archivos de un banco dado
    '''
    ruta = f'./Data/{banco}/'
    pdfs = [ruta + i for i in os.listdir(ruta)]
    d = {}
    for i in pdfs:
        if white_list:
            white_list.sort()
            if i.split('-')[0].split('/')[-1] in white_list:
#                 print(i)
                resultado = encontrar_key_word(keyword, keyword_black_list, i, verbose)
                d.setdefault(list(resultado.keys())[0],list(resultado.values())[0])
        else:
            resultado = encontrar_key_word(keyword, i)
            d.setdefault(list(resultado.keys())[0],list(resultado.values())[0], verbose)
    return d

In [9]:
def procesar_tablas(diccionario, banco):
    '''
    Funcion para juntar la informacion (diccionario) de todos los pdf que coincidieron con una keyword dada
    '''
    pagina = list(diccionario[list(diccionario.keys())[0]].keys())[0]
    df = None
    l = []
    for file in diccionario.keys():
        try:
            l += [diccionario[file][pagina].transpose()]
        except:
            pass
    df = l[0]
    l.remove(l[0])
    if len(l):
        for i in range(len(l)):
            df = df.join(l[i], how='right',lsuffix='_caller', rsuffix='_other').transpose().reset_index(drop=True)
            df = df.drop_duplicates(subset=list(df.columns)[0], keep='last')
        return df
    return df.transpose()

In [10]:
def limpiar_tabla(df):
    duplicated = list(df.columns[df.columns.duplicated()])
    special = 'Ingresos por Intereses Gastos por Intereses Comisiones cobradas '
    df.drop(columns=duplicated,inplace=True)
    ## Esta columna se lee como una sola cuando en realidad son 3, dividiremos la columna en sus respectivos campos
    if special in df.columns:
        vals = df[special].apply(lambda x: str(x).split(' '))
        vals_0 = [vals[i][0] for i in range(len(vals))]
        vals_1 = [vals[i][1] for i in range(len(vals))]
        vals_2 = [vals[i][2] for i in range(len(vals))]
        df['Ingresos por Intereses'] = vals_0
        df['Gastos por Intereses'] = vals_0
        df['Comisiones cobradas'] = vals_0
        df.drop(columns= [special], inplace=True)
        
    ## Limpiamos los valores para que tengan un formao numerico adecuado
    for i in df.columns:
        try:
            df[i] = df[i].astype(float)
        except:
            df[i] = df[i].apply(lambda x: str(x).strip().replace('(','').replace(')','').replace(',','').replace('-',''))
            
    ## Le damos formato a los nombres de las columnas
    df.colums = [str(i).strip() for i in list(df.columns)]
    cols = list(df.columns)
    cols = [str(i)[:-1] for i in cols if str(i)[-1]]
    df.columns = cols
    
    ## La O se escapa de nuestros anteriores filtros y lo tratamos nuevamenta
    ## Adicinalmente nos volvemos a asegurar que no existan espacion en los valores
    for c in df.columns:
        df[c] = [str(i).strip().replace(' ','') if i!='O' else 0 for i in df[c].values]
    return df

In [43]:
def encontrar_key_word(keyword, black_list_keywords, file, verbose=False):
    '''
    Funcion para encontrar la pagina con la lista de keywords dada 
    Devuelve un diccionario con las paginas que coinciden y su respectiva tabla
    '''
    numeros = []
    resultado = {}
    paginas = ['-'.join((file + f'/{i}').split('-')[:-1]).replace('-text','') for i in os.listdir(file)]
    archivos = [(i+'-tables.csv', i+'-inreadingorder.txt') if 'text' in i else (i+'-tables.csv', i+'-text-inreadingorder.txt') for i in paginas]
    name = archivos[0][0][:archivos[0][0].find('-pdf')].strip()
    for table, text in archivos:
        with open(text, 'r') as file:
            f = file.read()
        if sum([k in f for k in keyword]) == len(keyword) and not sum([k in f for k in black_list_keywords]):
#             print(f[:f.find(keyword)+len(keyword)])
#             print('#'*60)
            i = text.find('page-') + len('page-')
            try:
                numero = int(text[i:i+3])
            except:
                try:
                    numero = int(text[i:i+2])
                except:
                    numero = int(text[i:i+1])
            if numero < 4:
                continue
            tabla = hay_tabla(table, verbose)
            if type(tabla) != str:
                resultado.setdefault(f'Pagina {numero}', hay_tabla(table))
                numeros += [numero]
    numeros = list(set(numeros))
    resultado = {k: v for k, v in resultado.items() if type(v) != None and type(k) != None}
    temp = {name: resultado}
    return {k: v for k, v in temp.items() if type(v) != None and type(k) != None}

In [44]:
bbva_dict ={'Ingresos por intereses':'ingresos_interes','Gastos por intereses':'gastos_intereses','Ingresos por primas (neto)':'ingresos_primas','Incremento neto de reservas técnicas': 'inc_net_reservas_tecnicas','Siniestralidad, reclamaciones y otras obligaciones contractuales (neto)':'siniestralidad', 'Margen financiero':'marg_fin','Estimación preventiva para riesgos crediticios':'estim_preventiva_riesgos_cred','Margen financiero ajustado por riesgos crediticios':'marg_fin_riesgos_cred','Comisiones y tarifas cobradas':'comision_tarifacobrada','Comisiones y tarifas pagadas':'comision_tarifapagada','Resultado por intermediación':'resultado_intermediacion','Otros ingresos (egresos) de la operación':'ingresos_operacion','Gastos de administración y promoción':'gastos_admon_promo','Resultado de la operación':'resultado_operacion','Participación en el resultado de subsidiarias no consolidadas y asociadas':'resul_no_consoli_asociada','Resultado antes de impuestos a la utilidad':'resul_antes_impues_util','Impuestos a la utilidad causados':'impuesto_util_causados','Impuestos a la utilidad diferidos (netos)':'impuesto_util_diferidos','Participación en el resultado de subsidiarias y asociadas':'result_subsidia_asociada','Resultado antes de operaciones discontinuadas':'result_antes_opera_disconti','Operaciones discontinuadas':'operacion_discontinuada','Participación no controladora':'particip_no_controladora','Resultado neto':'resultado_neto'}
santander_dict = {'Ingresos por intereses':'ingresos_interes','Gastos por intereses':'gastos_intereses','Ingresos por primas (neto)':'ingresos_primas','Incremento neto de reservas técnicas': 'inc_net_reservas_tecnicas','Siniestralidad, reclamaciones y otras obligaciones contractuales (neto)':'siniestralidad', 'Margen financiero':'marg_fin','Estimación preventiva para riesgos crediticios':'estim_preventiva_riesgos_cred','Margen financiero ajustado por riesgos crediticios':'marg_fin_riesgos_cred','Comisiones y tarifas cobradas':'comision_tarifacobrada','Comisiones y tarifas pagadas':'comision_tarifapagada','Resultado por intermediación':'resultado_intermediacion','Otros ingresos de la operación':'ingresos_operacion','Gastos de administración y promoción':'gastos_admon_promo','Resultado de la operación':'resultado_operacion','Participación en el resultado de asociadas':'resul_no_consoli_asociada','Resultado antes de impuestos a la utilidad':'resul_antes_impues_util','Impuestos a la utilidad causados':'impuesto_util_causados','Impuestos a la utilidad diferidos (neto)':'impuesto_util_diferidos','Participación en el resultado de subsidiarias y asociadas':'result_subsidia_asociada','Utilidad neta mayoritaria':'result_antes_opera_disconti','Operaciones discontinuadas':'operacion_discontinuada','Participación no controladora':'particip_no_controladora','Utilidad neta mayoritaria':'resultado_neto'}
banorte_dict = {'Ingresos por Interese':'ingresos_interes','Gastos por Interese':'gastos_intereses','Ingresos por Primas (Neto)':'ingresos_primas','Incremento neto de reservas técnicas': 'inc_net_reservas_tecnicas','Siniestros, Reclamaciones y Otras Obligaciones':'siniestralidad', 'Ingresos de Intereses Netos antes de Estim. Ptva. para Riesgos Cred.':'marg_fin','Provisiones Prev. para Riesgos crediticios':'estim_preventiva_riesgos_cred','Ingresos de Intereses Netos Ajustado por Riesgos Crediticios':'marg_fin_riesgos_cred','Comisiones por Servicios Cobrados':'comision_tarifacobrada','Comisiones por Servicios Pagados':'comision_tarifapagada','Ingresos por Intermediación':'resultado_intermediacion','Total Otros Ingresos (Egresos) de la Operación':'ingresos_operacion','Total Gasto No Financiero':'gastos_admon_promo','Resultado de la Operación':'resultado_operacion','Participación en subsidiarias y asociadas no consolidadas':'resul_no_consoli_asociada','Resultados Antes de Impuestos a la Utilidad':'resul_antes_impues_util','Impuestos a la utilidad causados Impuesto al Activo':'impuesto_util_causados','Impuestos a la utilidad diferidos':'impuesto_util_diferidos','Participación en el resultado de subsidiarias y asociadas':'result_subsidia_asociada','Resultados antes de operaciones discontinuadas':'result_antes_opera_disconti','Operaciones discontinuadas Participación no controladora':'particip_no_controladora','Utilidad Neta':'resultado_neto'}
all_dicts = {'BBVA':bbva_dict, 'Santander':santander_dict, 'Banorte':banorte_dict}

In [45]:
list(all_dicts.keys())

['BBVA', 'Santander', 'Banorte']

In [50]:
def ab_team(params: dict, verbose=False):
    '''
    Params: diccionario con toda la informacion necesaria para encontrar la informacion en todos los bancons
    '''
    dfs = {}
    for key, value in params.items():
        if key != 'dicts':
#             try:
                dfs.setdefault(key, m_f(value['keywords'], value['black_list'], key, copy.copy(value['white_list']),verbose))
#             except Exception as e:
#                 print('No se encontro ninguna coincidencia, prueba a usar menos criterios en black list y mas en keywords')
#                 print(e)
                
    all_dicts = [santander_dict, banorte_dict, bbva_dict]
    
    
    for banco, df in dfs.items():
        df.rename(columns=params['dicts'][banco],inplace=True)

    for i in range(len(dfs)-1):
        if i == 0:
            data_merged = dfs[list(dfs.keys())[i]].append(dfs[list(dfs.keys())[i+1]], ignore_index=True)
        else:
            data_merged = data_merged.append(dfs[list(dfs.keys())[i+1]], ignore_index=True)
    #####Hacer rbind a las tablas

    dic = [list(i.values()) for i in all_dicts][0] + ['periodos', 'Banc']
    cols = list(data_merged.columns)
    data_merged = data_merged[[i for i in cols if i in dic]]
    return data_merged

In [16]:
params = {
    'BBVA': 
    {
     'keywords':['Estado de Resultados Consolidado'],
     'black_list': [],
     'white_list': ['1T21', '2T21']  
    },
    
    'Santander':
    {
     'keywords':['Estado de resultados consolidado'],
     'black_list': [],
     'white_list': ['2T21']          
    },
    
    'Banorte':
    {
     'keywords':['Estado de Resultados-GFNorte'],
     'black_list': [],
     'white_list': ['2T21_Reporte_trimestral']          
    },
    
    'dicts': all_dicts
}

In [51]:
df = ab_team(params)

<ipython-input-10-9f4e687908d0>:24: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.colums = [str(i).strip() for i in list(df.columns)]


In [52]:
df

,periodos,ingresos_interes,ingresos_primas,gastos_intereses,inc_net_reservas_tecnicas,siniestralidad,marg_fin,estim_preventiva_riesgos_cred,marg_fin_riesgos_cred,comision_tarifacobrada,...,gastos_admon_promo,resultado_operacion,resul_no_consoli_asociada,resul_antes_impues_util,impuesto_util_causados,impuesto_util_diferidos,result_subsidia_asociada,particip_no_controladora,resultado_neto,Banc
0,1T20,54302,7155,17546,238,7714,36435,16237,20198,11142,...,16683,12657,28.0,12685,6282,2215,19.0,2,8620,BBVA
1,2T20,43526,5043,15153,1859,6186,29089,7613,21476,9116,...,16237,14694,1.0,14695,701,2803,20.0,2,11189,BBVA
2,3T20,47619,5987,11991,341,7229,34045,7008,27037,10572,...,16317,20731,22.0,20753,8120,1619,42.0,0,14252,BBVA
3,4T20,46293,6532,10599,1011,7323,35914,16270,19644,12102,...,16853,12284,18.0,12302,5333,1829,0.0,1,8799,BBVA
4,1T21,46996,9395,10123,2617,8892,34759,10450,24309,11175,...,17409,15237,133.0,15370,3514,769,0.0,0,11087,BBVA
5,1T21,46996,9395,10123,2617,8892,34759,10450,24309,11175,...,17409,15237,133.0,15370,3514,769,0.0,0,11087,BBVA
6,2T21,25098,NaN,9328,NaN,NaN,15770,5068,10702,6662,...,9955,5895,50.0,5945,373,859,NaN,NaN,4713,Santander
7,1T21,25099,NaN,9514,NaN,NaN,15585,7075,8510,6535,...,9894,4252,77.0,4329,1151,101,NaN,NaN,3279,Santander
8,4T20,26541,NaN,10269,NaN,NaN,16272,3152,13120,6201,...,11102,7071,79.0,7150,1065,605,NaN,NaN,5480,Santander
9,3T20,27503,NaN,11414,NaN,NaN,16089,4596,11493,5659,...,10429,6637,60.0,6697,1364,303,NaN,NaN,5030,Santander
